# Gerekli Kütüphaneler

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier as SKMLPClassifier

seed = 42
np.random.seed(seed)

# Veri Seti

In [2]:
breast_cancer = datasets.load_breast_cancer()

In [3]:
X, y = breast_cancer.data, breast_cancer.target

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [27]:
y_train = [[_] for _ in y_train]
y_test = [[_] for _ in y_test]

# MLP

In [14]:
class MLP:
    def __init__(self, input_size, hidden_size, output_size, lr=0.01):
        # Ogrenme orani
        self.lr = lr
        # Input -> Hidden katmani agirlik ve bias
        self.W1 = np.random.randn(input_size, hidden_size)
        self.b1 = np.zeros((1, hidden_size))
        # Hidden -> Output katmani agirlik ve bias
        self.W2 = np.random.randn(hidden_size, output_size)
        self.b2 = np.zeros((1, output_size))

    # Sigmoid Aktivasyon Fonksiyonu
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    # Sigmoid Aktivasyon Fonksiyonu Turevi
    def sigmoid_derivative(self, z):
        return z * (1 - z)

    # Ileri besleme
    def forward(self, X):
        # Input -> Hidden
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        # Hidden -> Output
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.output = self.sigmoid(self.z2)
        return self.output

    # Geri yayilim
    def backward(self, X, y, output):
        # Output katmani hatasi
        output_error = y - output
        output_delta = output_error * self.sigmoid_derivative(output)

        # Hidden katmani hatasi
        hidden_error = np.dot(output_delta, self.W2.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.a1)

        # Agirliklari guncelleme
        self.W2 += self.lr * np.dot(self.a1.T, output_delta)
        self.b2 += self.lr * np.sum(output_delta, axis=0, keepdims=True)

        self.W1 += self.lr * np.dot(X.T, hidden_delta)
        self.b1 += self.lr * np.sum(hidden_delta, axis=0, keepdims=True)

    def train(self, X, y, n_iters):
        for i in range(n_iters):
            output = self.forward(X)
            self.backward(X, y, output)

    def predict(self, X):
        output = self.forward(X)
        return np.round(output)

# Eğitim

In [32]:
mlp = MLP(input_size=30, hidden_size=2, output_size=1, lr=0.01)
mlp.train(X_train, y_train, n_iters=10000)

# Tahmin

In [33]:
y_pred = mlp.predict(X_test)

# Sonuçlar

In [34]:
print("Accuracy score:", accuracy_score(y_test, y_pred))

Accuracy score: 0.6228070175438597


# Scikit-Learn

In [36]:
mlp2 = SKMLPClassifier(hidden_layer_sizes=2, learning_rate="constant", learning_rate_init=0.01)
mlp2.fit(X_train, y_train)

/home/alper/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(hidden_layer_sizes=2, learning_rate_init=0.01)

In [37]:
y_pred_2 = mlp2.predict(X_test)

In [38]:
print("Accuracy score:", accuracy_score(y_test, y_pred_2))

Accuracy score: 0.6228070175438597
